* Ссылки 
  * [Гайд по SNS](https://nagornyy.me/courses/data-science/intro-to-seaborn/).<br>
  * [Построение графиков](https://pcnews.ru/blogs/spargalka_po_vizualizacii_dannyh_v_python_s_pomosu_plotly-986119.html).<br>
  * [Продвинутая визуализация](https://habr.com/ru/company/skillfactory/blog/510320/).<br>
  * [Все графики](https://prog-help.ru/python/kollekcija-prodvinutoj-vizualizacii-v-matplotlib-i-seaborn-s-primerami/).<br>

In [ ]:
from pylab import rcParams
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import warnings
import numpy as np
import pandas as pd
import plotly
from IPython.display import Image
import matplotlib
from IPython.core.interactiveshell import InteractiveShell
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
%matplotlib inline
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 16, 10
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)
init_notebook_mode(connected=True)
pd.options.display.float_format = '{:,.1f}'.format
InteractiveShell.ast_node_interactivity = 'all'

df = pd.read_csv('df.csv')

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('precision', 2)

In [ ]:
df.columns = ['SEQN', 'Alcohol101', 'Alcohol110', 'Alcohol130', 'Smoking', 'Gender',
       'Age', 'Race', 'Country', 'Education', 'Family', 'HouseSize',
       'Salary', 'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BloodPressureSY1', 'BloodPressureDI1', 'BloodPressureSY2',
       'BloodPressureDI2', 'Weight', 'Height', 'BodyIndex', 'BodyLeg', 'BodyArm', 'BodyArm2',
       'BodyWaist', 'HIQ210']

In [ ]:
df.head()

In [ ]:
# Преобразуем числовые данные в объектные
df['Gender'] = df.Gender.replace({1: "Male", 2: "Female"})
df['Education'] = df.Education.replace(
    {1: "<9", 2: "9-11", 3: "HS/GED", 4: "Some college/AA", 5: "College", 7: "Refused", 9: "Don't know"})
df['Family'] = df.Family.replace({1: "Married", 2: "Widowed", 3: "Divorced",
                                  4: "Separated", 5: "Never married", 6: "Living w/partner", 77: "Refused"})

In [ ]:
# Добавим новый столбец, с массой тела, умноженной на 10
df["dot_size"] = df.BodyIndex*10

In [ ]:
# Уменьшим фрейм
df = df.loc[:500, :]
df.shape

In [ ]:
# Диаграмма рассеяния
from scipy.spatial import ConvexHull

# Функция, которая добавляет в график очерченную область, где переменные предикторов соответствуют определенному критерию
from scipy.spatial import ConvexHull
def encircle(x, y, ax=None, **kw):
    if not ax:
        ax = plt.gca()
    p = np.c_[x, y]
    hull = ConvexHull(p)
    poly = plt.Polygon(p[hull.vertices, :], **kw)
    ax.add_patch(poly)

category = df['Family'].unique()

plt.figure(figsize=(16, 8), dpi=80, facecolor="w", edgecolor="k")
for i, cat in enumerate(category):
    plt.scatter("BloodPressureDI1", "BloodPressureSY1",  # X Y
                # В каждом цикле проходим по одному значению переменных из колонки
                data=df.loc[df.Family == cat, :],
                s='dot_size',  # ЗАвисимость размера точки от значения в другой колонке фрейма
                label=str(cat))

# Задаем целевой предиктор
df_encircle = df.loc[(df["Age"] > 40), :].dropna()
# Ограничиваем область и делаем ей заливку
encircle(df_encircle.BloodPressureDI1, df_encircle.BloodPressureSY1, ec="k", fc="gold", alpha=0.1)
encircle(df_encircle.BloodPressureDI1, df_encircle.BloodPressureSY1,
         ec="firebrick", fc="none", linewidth=1.5)

plt.gca().set(xlabel='BloodPressureDI1', ylabel='BloodPressureSY1')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Bubble Plot with Encircling", fontsize=22)
plt.legend(fontsize=12)
plt.show();

In [ ]:
# точечная диаграмма
fig, ax = plt.subplots(figsize=(16, 8), dpi=80)
sns.stripplot(df.BloodPressureSY1, df.BodyIndex, jitter=0.45, size=8, ax=ax, linewidth=0.5)
plt.title("Systolic Blood pressure vs Body mass index")
plt.tick_params(axis='x', which='major', labelsize=12, rotation=90)
plt.show();

In [ ]:
# Точечные диаграммы с ящиками помогают когда данных очень много
fig, ax = plt.subplots(figsize=(30, 12))
ax = sns.boxplot(x="BloodPressureDI1", y = "BloodPressureSY1", data=df)
ax.tick_params(rotation=90, labelsize=18)
ax = sns.stripplot(x = "BloodPressureDI1", y = "BloodPressureSY1", data=df)

In [ ]:
# Точечная диаграмма, разделенная по дополнительному предиктору (hue)
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)    
sns.stripplot(df.BloodPressureDI1, df.BloodPressureSY1, s=10, hue = df.Race, ax=ax)
plt.title("Stripplot for Systolic vs Diastolic Blood Pressure", fontsize=20)
plt.tick_params(rotation=90)
plt.show();

In [ ]:
#Скрипичная диаграмма
fig, ax = plt.subplots(figsize=(30, 12))
ax = sns.violinplot(x= "Family", y="Age", data=df, inner=None, color="0.4")
ax = sns.stripplot(x= "Family", y="Age", data=df)
ax.tick_params(rotation=90, labelsize=28)

In [ ]:
# Скрипичная диаграмма зависимости отклика от предиктора, с разделением по второму предиктору (hue) 
fig = plt.figure(figsize=(16, 8), dpi=80)

# Добавление второго графика, справа
grid=plt.GridSpec(4, 4, hspace=0.5, wspace=0.2)
ax_main = fig.add_subplot(grid[:, :-1])
ax_right = fig.add_subplot(grid[:, -1], xticklabels=[], yticklabels=[])

sns.violinplot(x= "Family", y = "BloodPressureDI1", hue = "Gender", data = df, color= "0.2", ax=ax_main)
sns.stripplot(x= "Family", y = "BloodPressureDI1", data = df, ax=ax_main)

ax_right.hist(df.BloodPressureDI1, histtype='stepfilled', orientation='horizontal', color='grey')
ax_main.title.set_fontsize(14)
ax_main.tick_params(rotation=10, labelsize=14)

plt.show();

In [ ]:
# Диаграммы с линией линейной регрессии. Зависимость отклика от предиктора, разделенных по доп. предиктору

g = sns.lmplot(x='Height', y='Weight', hue='Gender', data=df,
               aspect=1.5, robust=True, palette='tab10',
               scatter_kws=dict(s=60, linewidths=.7, edgecolors='black'))
plt.title("Height vs weight with line of best fit grouped by Gender", fontsize=20)
plt.show()

In [ ]:
# Индивидуальные диаграммы с линией регрессии. Отклик и предиктор с доп. графиками для второго предиктора

fig = plt.figure(figsize=(20, 8), dpi=80)
g = sns.lmplot(x='Height', y='Weight', data = df, robust = True,
              palette="Set1", col="Race", # Указываем на разделение на колонки по параметрам в доп. предикторе
              scatter_kws=dict(s=60, linewidths=0.7, edgecolors="black"))
plt.xticks(fontsize=12, )
plt.yticks(fontsize=12)
plt.show();

In [ ]:
# Парные диаграммы для EDA.Взаимозависимости 4 пар предикторов, разделенных по пятому
plt.figure(figsize=(10,8), dpi= 80)
sns.color_palette("hls", 8)
sns.pairplot(df[['Weight', 'Height', 'BodyIndex', 'BodyWaist', "Race"]], kind="scatter", hue="Race", plot_kws=dict(s=30))
plt.show();

In [ ]:
# Расходящиеся столбики

x = df.loc[:, "BloodPressureSY1"]
df["BloodPressureSY1_n"] = (x - x.mean())/x.std()  # Стандартизируем данные
# Задаем цвета для положительных и отрицательных значений
df['colors'] = ['red' if i < 0 else 'blue' for i in df["BloodPressureSY1_n"]]
df.sort_values("BloodPressureSY1_n", inplace=True)
df.reset_index(inplace=True)
plt.figure(figsize=(16, 10), dpi=80)
plt.hlines(y=df.HouseSize, xmin=0, xmax=df.BloodPressureSY1_n,
           color=df.colors, linewidth=3)  # Центрируем по стандартизированным
plt.gca().set(ylabel="HouseSize", xlabel="BPXSY1_n")
plt.yticks(df.HouseSize, fontsize=14)
plt.grid(linestyle='--', alpha=0.5)
plt.show();

In [ ]:
# Расходящиеся столбики
x = df.loc[:, "BloodPressureSY1"]
df['colors'] = ['coral' if i < 0 else 'lightgreen' for i in df["BloodPressureSY1_n"]]
y_ticks = np.arange(16, 82, 8) #Задаем шкалу для оси Y. старт, стоп, шаг
plt.figure(figsize=(16, 10), dpi=80)
plt.hlines(y = df.Age, xmin=0, xmax = df.BloodPressureSY1_n, color=df.colors, linewidth=3)
plt.gca().set(ylabel="Age", xlabel = "BloodPressureSY1")
plt.yticks(y_ticks, fontsize=14)
plt.grid(linestyle='--', alpha=0.5)
plt.show();